In [1]:
import torch
import torch.nn
import numpy as np

a = np.random.randint(0, 10, 5)
a = torch.tensor(a)
a

tensor([0, 2, 7, 2, 2])

In [2]:
b = torch.tensor(np.random.randint(0, 10, 5))
b

tensor([7, 5, 0, 8, 0])

In [3]:
c = torch.add(a, b)
c_summed = torch.sum(c)

c, c_summed

(tensor([ 7,  7,  7, 10,  2]), tensor(33))

In [4]:
d = torch.tanh(b.float())
d

tensor([1.0000, 0.9999, 0.0000, 1.0000, 0.0000])

In [5]:
torch.mean(d)

tensor(0.6000)

In [6]:
import tensorflow as tf
import numpy as np

f = tf.constant(np.random.randint(0, 10, 5), shape=(1, 5), dtype=tf.int32)
print(f)
g = tf.constant(np.random.randint(0, 10, 5), shape=(1, 5), dtype=tf.int32)
print(g)
result = A + B
tanh = tf.tanh(tf.to_float(result))


with tf.Session() as sess:
    print("f: {}    g: {}\n".format(sess.run(f), sess.run(g)))
    print("f + g: {}\n".format(sess.run(result)))
    print("tanh(f+g): {}".format(sess.run(tanh)))

2023-02-22 00:15:39.584961: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


tf.Tensor([[3 2 6 6 3]], shape=(1, 5), dtype=int32)
tf.Tensor([[8 3 6 7 0]], shape=(1, 5), dtype=int32)


2023-02-22 00:15:52.076931: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


NameError: name 'A' is not defined

In [79]:
### Example on MNIST

In [12]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

#device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = 'cpu'

In [8]:

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dense1 = nn.Linear(12*12*64, 128)
        self.dense2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.dropout(x, 0.25)
        x = x.view(-1, 12*12*64)
        x = F.relu(self.dense1(x))
        x = F.dropout(x, 0.5)
        x = self.dense2(x)
        return F.log_softmax(x, dim=1)



In [9]:
def train(model, device, train_loader, criterion, optimizer, epoch, save_dir='model.ckpt'):
    total_step = len(train_loader)
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
    torch.save(model.state_dict(), 'pytorch_mnist_cnn.ckpt')

In [10]:
from sklearn.metrics import roc_auc_score

def test(model, device, test_loader):
    
    preds = []
    y_true = []
    # Test the model
    model.eval()  # Set model to evaluation mode.
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()    
            detached_pred = predicted.detach().cpu().numpy()
            detached_label = labels.detach().cpu().numpy()
            for f in range(0, len(detached_pred)):
                preds.append(detached_pred[f])
                y_true.append(detached_label[f])

        print('Test Accuracy of the model on the 10000 test images: {:.2%}'.format(correct / total))

        preds = np.eye(num_classes)[preds]
        y_true = np.eye(num_classes)[y_true]    
        auc = roc_auc_score(preds, y_true)
        print("AUC: {:.2%}".format (auc))
        

In [11]:
#### Hyperparameters
num_epochs = 15
num_classes = 10
batch_size = 128
learning_rate = 0.001

# Load MNIST data set
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)



model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


## Training phase

for epoch in range(0, num_epochs):
    train(model, device, train_loader, criterion, optimizer, epoch)
    
## Testing phase


test(model, device, test_loader)

Epoch [1/15], Step [100/469], Loss: 0.2640
Epoch [1/15], Step [200/469], Loss: 0.1597
Epoch [1/15], Step [300/469], Loss: 0.1254
Epoch [1/15], Step [400/469], Loss: 0.0510
Epoch [2/15], Step [100/469], Loss: 0.0862
Epoch [2/15], Step [200/469], Loss: 0.0654
Epoch [2/15], Step [300/469], Loss: 0.1176
Epoch [2/15], Step [400/469], Loss: 0.1426
Epoch [3/15], Step [100/469], Loss: 0.0438
Epoch [3/15], Step [200/469], Loss: 0.0462
Epoch [3/15], Step [300/469], Loss: 0.0755
Epoch [3/15], Step [400/469], Loss: 0.1518
Epoch [4/15], Step [100/469], Loss: 0.0198
Epoch [4/15], Step [200/469], Loss: 0.0836
Epoch [4/15], Step [300/469], Loss: 0.0723
Epoch [4/15], Step [400/469], Loss: 0.0624
Epoch [5/15], Step [100/469], Loss: 0.0445
Epoch [5/15], Step [200/469], Loss: 0.0584
Epoch [5/15], Step [300/469], Loss: 0.0313
Epoch [5/15], Step [400/469], Loss: 0.0630
Epoch [6/15], Step [100/469], Loss: 0.0721
Epoch [6/15], Step [200/469], Loss: 0.0480
Epoch [6/15], Step [300/469], Loss: 0.1003
Epoch [6/15

In [ ]:
## sequential
import torch.nn as nn

model = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, 1),
            nn.Sigmoid()
            )

## ModuleList

class ModuleListModel(nn.Module):
    def __init__(self):
        super(ModuleListModel, self).__init__()
        self.conv_1 = nn.Conv2d(1, 32, 3, 1)
        self.conv_2 = nn.Conv2d(32, 64, 3, 1)
        self.dense_1 = nn.Linear(64*64, 128)
        self.output = nn.Linear(128, n_classes)
        
        
    def forward(self, x):
        x = nn.functional.relu(self.conv_1(x))
        x = nn.functional.relu(self.conv_2(x))
        x = nn.functional.max_pool2d(x, 2, 2)
        x = nn.functional.dropout(x, 0.25)
        x = x.view(-1, 64*64)
        x = nn.functional.reu(self.dense_1(x))
        x = nn.functional.dropout(x, 0.5)
        x = nn.functional.log_softmax(self.output(x), dim=1)
        return x
    

model = ModuleListModel().to(device)

In [ ]:
def forward(self, x):
    x = nn.functional.relu(self.conv_1(x))
    x = nn.functional.dropout(x, 0.25)
    x = nn.functional.relu(self.conv_2(x))
    ...

In [ ]:
def forward(self, x):
    ...
    x = nn.functional.dropout(x, 0.5)
    x = nn.functional.softmax(self.dense_1(x), dim=1)
    return x
    

In [ ]:
def forward(self, x):
    ...
    x = nn.functional.dropout(x, 0.5)
    x = nn.functional.log_softmax(self.dense_1(x), dim=1)
    return x

In [ ]:
def forward(self, x):
    ...
    x = nn.functional.dropout(x, 0.5)
    x = nn.functional.sigmoid(self.dense_1(x), dim=1)
    return x